This file has the results of testing the effects of tolerance on my adaptive solver in order to see how the algorithm can be sped up without negative effects.

In [4]:
using BenchmarkTools
using CSV
using DataFrames
using Random
using Distributions
using Plots
include("EvaluationFunctions.jl")
include("Constants.jl")

900

In [32]:
psym = [:ka1 => 0.009433439939827041, :kb1 => 2.3550169939427845, :kcat1 => 832.7213093872278, :ka2 => 12.993995997539924, :kb2 => 6.150972501791291,
        :ka3 => 1.3481451097940793, :kb3 => 0.006201726090609513, :ka4 => 0.006277294665474662, :kb4 => 0.9250191811994848, :ka7 => 57.36471615394549, 
        :kb7 => 0.04411989797898752, :kcat7 => 42.288085868394326, :y => 3631.050539219606]
p = [x[2] for x in psym]
    
#initial condition list
usym = [:L => 0, :K => 10^-0.2895987, :P => 0.820348, :A => 10^0.42483, :Lp => 0.0, :LpA => 0.0, :LK => 0.0, 
        :LpP => 0.0, :LpAK => 0.0, :LpAP => 0.0, :LpAKL => 0.0, :LpAPLp => 0.0, :AK => 0.0, :AP => 0.0, 
        :AKL => 0.0, :APLp => 0.0]
u0 = [x[2] for x in usym]

16-element Vector{Float64}:
 0.0
 0.5133355009762635
 0.820348
 2.6596837508207485
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Below is some modified code from when I was testing the adaptive solver as a classifier. It is useful for iterating the adaptive solver over many  different initial concentrations and generating output that cna then be analyzed to ensure that the number of numerical errors is not increasing.

In [33]:
function testClassifier(numIterations; saveCSV = true, outputdirectory = "/Users/ezragreenberg/Julia/Someplots/",filename="mytest1.csv", abstol=1e-8, reltol=1e-12)
    #df = DataFrame(u0=Vector{Float64}[], p=Vector{Float64}[], retcode=Float64[], per=Float64[], amp=Float64[])
    #CSV of array gets read in as string, will have to generalize this later
    df = DataFrame(L=Float64[],K=Float64[],P=Float64[],A=Float64[], retcode=Float64[], per=Float64[], amp=Float64[])
    for i in 1:numIterations
        u0[1] = rand(Random.seed!(i),Distributions.LogUniform(0.01, 100)) #Lp #1 for L, 2 for Lp
        u0[2] = rand(Random.seed!(numIterations + i),Distributions.LogUniform(0.001, 100)) #K
        u0[3] = rand(Random.seed!(2 * numIterations + i),Distributions.LogUniform(0.01, 100)) #P
        u0[4] = rand(Random.seed!(3 * numIterations + i),Distributions.LogUniform(0.001, 100)) #A
        retcode = adaptiveSolve(prob, u0, shortSpan, longSpan, p; abstol = abstol, reltol = reltol)
        push!(df, Dict(:L=>u0[1],:K=>u0[2],:P=>u0[3],:A=>u0[4], :retcode => retcode[1], :per => retcode[2], :amp =>retcode[3]))
    end
    if saveCSV
        CSV.write(outputdirectory*filename, df)
    end
    return df
end

testClassifier (generic function with 1 method)

I first will test it with the tolerances I have set so far: reltol of 1e-12 and abstol of 1e-8

In [34]:
@btime(testClassifier(10000; outputdirectory = "/Users/ezragreenberg/Julia/ExperimentalFullModelWork/BenchmarkCsv/", filename = "DefaultTolerance.csv"))

  12.556 s (5247777 allocations: 1.27 GiB)


Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0196547,0.302516,48.8004,0.104239,1.1,0.0,0.0
2,0.0102095,0.00388036,0.564983,0.00240571,2.0,0.0,0.0
3,0.228528,6.19631,28.7427,4.61154,1.1,0.0,0.0
4,26.1779,0.222948,0.765262,9.06722,1.1,0.0,0.0
5,81.3937,0.0146195,0.511188,0.0429213,2.0,0.0,0.0
6,21.876,0.592596,0.0215943,0.0856965,1.1,0.0,0.0
7,0.0944455,0.00169754,0.540428,0.0316907,2.0,0.0,0.0
8,0.485438,1.4489,0.0901489,0.202443,1.1,0.0,0.0
9,0.195302,0.700483,2.5215,0.0150069,1.1,0.0,0.0


Below we look at the number of numerical errors by looking at the number of outputs with a return code of 1.5 (indicating mass conservation failed) or 1.0 (indicating some other numerical failure).

In [35]:
mydf1 = DataFrame(CSV.File("/Users/ezragreenberg/Julia/ExperimentalFullModelWork/BenchmarkCsv/DefaultTolerance.csv"))
mass_not_conserved1 = mydf1[mydf1.retcode.==1.5,:]

Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [36]:
other_failures1 = mydf1[mydf1.retcode.==1.0,:]

Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64


And below we look at some oscillatory solutions

In [37]:
osc_df1  = mydf1[mydf1.retcode.<1.0,:]

Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,13.8915,0.588267,4.96728,2.0392,-100.0,2.9871,0.387939
2,8.60709,3.13347,1.35597,72.6048,-100.0,1.906,1.80404
3,7.44909,0.0471053,0.17936,1.06195,-100.0,4.58,0.115861
4,32.3703,0.48104,4.20911,6.12865,-100.0,1.87451,0.971606
5,53.1952,5.4238,35.4178,31.112,-1.0,1.43333,12.1741
6,22.7287,2.30622,13.678,11.0956,-1.0,1.67805,2.2754
7,21.4898,2.94398,42.6278,5.81731,-100.0,3.05161,0.412924
8,25.9791,2.08729,24.8918,8.50543,-100.0,1.97917,0.878368
9,4.42054,2.44321,0.877612,26.4603,-100.0,1.89216,0.942524


In [40]:
function entryToSol(df, row)
    currow = df[row,:]
    u0[1] = currow[:L]
    u0[2] = currow[:K]
    u0[3] = currow[:P]
    u0[4] = currow[:A]
    return solve(remake(prob, u0=u0, tspan=(0,longSpan), p=p), RadauIIA5(), abstol=1e-8, reltol=1e-12, saveat=0.1, save_idxs=1, maxiters=10000, verbose=false)
end

function PlotSolutions(osc_df, numsols = size(osc_df)[1])
    for i in 1:numsols
        cursol = entryToSol(osc_df, i)
        plot(cursol, title="L vs t")
        savefig("/Users/ezragreenberg/Julia/ExperimentalFullModelWork/BenchmarkCsv/Someplots/$i.png")
    end
end

PlotSolutions (generic function with 2 methods)

In [41]:
PlotSolutions(osc_df1)

All 50 solutions that are pulled out are oscillatory. We had no numerical errors (other than reaching max iterations). Let's now try with the default tolerance of Julia, 1e-8 for reltol and 1e-6 for abstol

In [44]:
@btime(testClassifier(10000; outputdirectory = "/Users/ezragreenberg/Julia/ExperimentalFullModelWork/BenchmarkCsv/", filename = "LowerTolerance.csv", reltol=1e-6, abstol=1e-8))

  8.243 s (4112921 allocations: 1.06 GiB)


Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0196547,0.302516,48.8004,0.104239,1.1,0.0,0.0
2,0.0102095,0.00388036,0.564983,0.00240571,2.0,0.0,0.0
3,0.228528,6.19631,28.7427,4.61154,1.1,0.0,0.0
4,26.1779,0.222948,0.765262,9.06722,1.1,0.0,0.0
5,81.3937,0.0146195,0.511188,0.0429213,2.0,0.0,0.0
6,21.876,0.592596,0.0215943,0.0856965,1.1,0.0,0.0
7,0.0944455,0.00169754,0.540428,0.0316907,2.0,0.0,0.0
8,0.485438,1.4489,0.0901489,0.202443,1.1,0.0,0.0
9,0.195302,0.700483,2.5215,0.0150069,1.1,0.0,0.0


In [46]:
mydf2 = DataFrame(CSV.File("/Users/ezragreenberg/Julia/ExperimentalFullModelWork/BenchmarkCsv/LowerTolerance.csv"))
mass_not_conserved2 = mydf1[mydf1.retcode.==1.5,:]

Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [47]:
other_failures2 = mydf1[mydf1.retcode.==1.0,:]

Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [48]:
osc_df2  = mydf2[mydf2.retcode.<1.0,:]

Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,13.8915,0.588267,4.96728,2.0392,-100.0,2.9871,0.387939
2,8.60709,3.13347,1.35597,72.6048,-100.0,1.906,1.80404
3,7.44909,0.0471053,0.17936,1.06195,-100.0,4.58,0.115861
4,32.3703,0.48104,4.20911,6.12865,-100.0,1.87451,0.971606
5,53.1952,5.4238,35.4178,31.112,-1.0,1.43019,12.0376
6,22.7287,2.30622,13.678,11.0956,-100.0,1.67719,2.26789
7,21.4898,2.94398,42.6278,5.81731,-100.0,3.05161,0.412925
8,25.9791,2.08729,24.8918,8.50543,-100.0,1.97917,0.878367
9,4.42054,2.44321,0.877612,26.4603,-100.0,1.89216,0.942524


In [49]:
PlotSolutions(osc_df2)

So we got the same results in this situation it appears. This should be explored further in the case of parameters that have been issues in the past; storing these values is a to do. Note however that there was a ~30% decrease in the time it took. Let's finish off by timing what happens if we do -8 and -8 as Dr. Johnson has suggested in the past.

In [50]:
@btime(testClassifier(10000; outputdirectory = "/Users/ezragreenberg/Julia/ExperimentalFullModelWork/BenchmarkCsv/", filename = "MidTolerance.csv", reltol=1e-8, abstol=1e-8))

  11.075 s (4883714 allocations: 1.20 GiB)


Row,L,K,P,A,retcode,per,amp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0196547,0.302516,48.8004,0.104239,1.1,0.0,0.0
2,0.0102095,0.00388036,0.564983,0.00240571,2.0,0.0,0.0
3,0.228528,6.19631,28.7427,4.61154,1.1,0.0,0.0
4,26.1779,0.222948,0.765262,9.06722,1.1,0.0,0.0
5,81.3937,0.0146195,0.511188,0.0429213,2.0,0.0,0.0
6,21.876,0.592596,0.0215943,0.0856965,1.1,0.0,0.0
7,0.0944455,0.00169754,0.540428,0.0316907,2.0,0.0,0.0
8,0.485438,1.4489,0.0901489,0.202443,1.1,0.0,0.0
9,0.195302,0.700483,2.5215,0.0150069,1.1,0.0,0.0


Ok, so this was not much of a time reduction from 1e-8 and 1e-12. So unless we can reduce to 1e-6, probably not worth doing.